## Import libraries

In [1]:
from pathlib import Path
import pandas as pd
import math
import openpyxl

# Create peak pressure file - Attempt 1 (without peak index)

## Access input folders

In [4]:
#Access input folder
input_dir1 = Path ("8series_tracers_pressure")
print ("1",input_dir1)

#Access folders inside input folder
input_dir2 =  [folder for folder in input_dir1.iterdir() if folder.is_dir()]
print ("2",input_dir2)

1 8series_tracers_pressure
2 [WindowsPath('8series_tracers_pressure/L803')]


## Create output files 1

In [ ]:
%%timeit -r1 -n1
# define current work directory
current_dir = Path.cwd()

# access the template to be used to generate files
excel_template = current_dir/ "Template.xlsx"

# Access each folder inside input folder
for folder in input_dir2:
    print ("folder", folder)
    
    # Make a list of data file names
    files = list(folder.rglob("*"))
    
    # Make a set of file names
    keys = set (file.stem for file in files)
    
    # Use xlwings library to create excel files
    with xw.App (visible = False) as app:
        
        # for loop for keys
        for key in keys:
            
            # Open template excel file
            wb = app.books.open(excel_template)
            
            # for loop for files
            for file in files:
                if file.stem == key:
                    print ("file.stem = ", file.stem)
                    print ("key = ", key)
                    df = pd.read_csv(file, delimiter="\s+", skiprows = 8, skipfooter = 1, header = None, names = ["time", "pressure"])
                    wb.sheets[0].range("A1").options(index=False).value = df
            output_dir = current_dir/ "Pressure_curves_excel"
            output_dir.mkdir(exist_ok = True)
            output_dir1 = output_dir/ folder.stem
            output_dir1.mkdir(exist_ok = True)
            wb.save(output_dir1/f"{key}.xlsx")
        #print ("file",folder.stem, file.stem)
        #current_dir = Path.cwd()
        #excel_template = current_dir/ "Template.xlsx"
        

## Create output files 2

In [6]:
df = pd.DataFrame(columns = ['Charge_mass', 'Standoff_distance', 'Peak_Pressure', 'Peak_time'])
print (df)

# Access each folder inside input folder
for folder in input_dir2:
    print ("folder", folder)
    charge_mass = int(folder.stem[2:])/2
    
    # Make a list of data file names
    files = list(folder.rglob("*"))
        
    # Access each file 
    for file in files:
        print ("file", file)
        stand_dis = int(file.stem[1:])/2
            
        # Create a dataframe out of data files and use delimiters
        df1 = pd.read_csv(file,
                          delimiter="\s+",
                          skiprows = 7,
                          nrows= 1,
                          engine = 'python',
                          header = None)
            
        # append rows to an empty DataFrame
        new_row = {'Charge_mass' : [charge_mass], 'Standoff_distance' : [stand_dis], 'Peak_Pressure' : [df1.iloc[0,2]], 'Peak_time' : [df1.iloc[0,5]]}
        new_row_data = pd.DataFrame(new_row)
        df = pd.concat([df, new_row_data], ignore_index = True)
print (df)     

Empty DataFrame
Columns: [Charge_mass, Standoff_distance, Peak_Pressure, Peak_time]
Index: []
folder 8series_tracers_pressure\L803
file 8series_tracers_pressure\L803\T11
file 8series_tracers_pressure\L803\T13
file 8series_tracers_pressure\L803\T15
file 8series_tracers_pressure\L803\T17
file 8series_tracers_pressure\L803\T19
file 8series_tracers_pressure\L803\T2
file 8series_tracers_pressure\L803\T21
file 8series_tracers_pressure\L803\T23
file 8series_tracers_pressure\L803\T25
file 8series_tracers_pressure\L803\T27
file 8series_tracers_pressure\L803\T29
file 8series_tracers_pressure\L803\T3
file 8series_tracers_pressure\L803\T31
file 8series_tracers_pressure\L803\T33
file 8series_tracers_pressure\L803\T35
file 8series_tracers_pressure\L803\T37
file 8series_tracers_pressure\L803\T39
file 8series_tracers_pressure\L803\T40
file 8series_tracers_pressure\L803\T5
file 8series_tracers_pressure\L803\T7
file 8series_tracers_pressure\L803\T9
    Charge_mass  Standoff_distance  Peak_Pressure  Peak

# Create peak pressure file - Attempt 2 (with peak index)

## Access input folders 2

In [ ]:
#Access input folder
input_dir3 = Path ("Pressure_curves_excel_with_MinMaxTime")
print ("3",input_dir3)

#Access folders inside input folder
input_dir4 =  [folder for folder in input_dir3.iterdir() if folder.is_dir()]
print ("4",input_dir4)

## Create peak pressure dataframe

In [ ]:
df1 = pd.DataFrame(columns = ['Charge_mass', 'Standoff_distance', 'Peak pressure', 'Peak time', 'Peak index'])
# Access each folder inside input folder
for folder in input_dir4:
    print ("folder", folder)
    charge_mass = int(folder.stem[4:])/2
    
    # Make a list of data file names
    files = list(folder.rglob("*"))
        
    # Access each file 
    for file in files:
        wrkbk = openpyxl.load_workbook(file)
        sh = wrkbk.active
        stand_dis = math.ceil(int(file.stem[1:])/2)
        peak_time = sh.cell(row = 2, column = 4)
        print ('file', file, 'peak_time', peak_time.value)
        
        # iterate through excel and display data
        for i in range(2, sh.max_row+1):
            cell_time = sh.cell (row = i, column = 1)
            #print (cell_time.value)
            if round(cell_time.value, 10) >= peak_time.value:
                print ("cell_time", cell_time.value)
                peak_index = i
                peak_pressure = sh.cell (row = i, column = 2)
                print ("peak_index", i, 'peak_pressure', peak_pressure.value)
                
                # append rows to an empty DataFrame
                new_row = {'Charge_mass' : [charge_mass],
                           'Standoff_distance' : [stand_dis],
                           'Peak pressure' : [peak_pressure.value],
                           'Peak time' : [peak_time.value],
                           'Peak index' : [i]}
                new_row_data = pd.DataFrame(new_row)
                df1 = pd.concat([df1, new_row_data], ignore_index = True)
                break     

In [ ]:
print (df1)

## Create peak pressure excel file

In [ ]:
with pd.ExcelWriter('Peak_Incident_Pressure_with_peak_index.xlsx', engine = "openpyxl", mode='a', if_sheet_exists='replace') as writer:  
    df1.to_excel(writer, sheet_name = "Sheet1", index = False, header = True)